In [1]:
import os
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import pandas as pd
import urllib.parse
import tempfile
from datetime import datetime

In [2]:
# Relleno de Nulos, Carga y Actualización de Archivos de Google Drive

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = r"C:\Users\danip\Desktop\AMC-Drinks\Analisis_Inventario_Distribuidora_Licores_AMCDrinks\Automatización\fair-gist-438316-e9-ecb638e26236.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive
folder_id = '1XPxR4idoWQxystA5J99X0ZkHn24cears'  # Cambia esto según tu carpeta

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a Google Drive
def subir_archivo(file_id, upload_file_path):
    media = MediaFileUpload(upload_file_path, resumable=True)
    drive_service.files().update(fileId=file_id, media_body=media).execute()
    print(f'Archivo actualizado en Google Drive: {upload_file_path}')

# Función para obtener la lista de archivos en la carpeta de Google Drive
def obtener_archivos_en_carpeta(folder_id):
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    return results.get('files', [])

# Obtener la lista de archivos en la carpeta de Google Drive
files = obtener_archivos_en_carpeta(folder_id)

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()
archivos_modificados = []

# Iterar sobre los archivos
for file in files:
    file_id = file['id']
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file_id, download_path)

    # Leer el archivo CSV y hacer modificaciones
    df = pd.read_csv(download_path)
    
    # Rellenar los valores nulos
    df.fillna("no definido", inplace=True)

    # Guardar los cambios en el archivo CSV
    df.to_csv(download_path, index=False)

    # Agregar a la lista de archivos modificados
    archivos_modificados.append((file_id, download_path))

# Subir los archivos modificados a Google Drive
for file_id, download_path in archivos_modificados:
    subir_archivo(file_id, download_path)

# Guardar la marca de tiempo de la última ejecución
marca_de_tiempo_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open("ultima_ejecucion.txt", "w") as file:
    file.write(marca_de_tiempo_actual)

print("Proceso completado.")

Descargado 100%.
Descargado 100%.
Descargado 77%.
Descargado 100%.
Descargado 100%.
Descargado 100%.
Descargado 100%.
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\INVENTARIO_INICIAL_TIENDA.csv
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\INVENTARIO_FINAL_TIENDA.csv
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\VENTAS_FINAL.csv
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\PRECIOS_PRODUCTO.csv
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\ORDENES_COMPRA.csv
Archivo actualizado en Google Drive: C:\Users\danip\AppData\Local\Temp\COMPRAS_FINAL_DETALLE.csv
Proceso completado.
